In [1]:
import requests
import pandas as pd 
from bs4 import BeautifulSoup


base_url = "https://www.consultant.ru"

In [2]:
response_base_url = requests.get(base_url)
soup = BeautifulSoup(response_base_url.text, 'lxml')

links = soup.select('ul.useful-links__list li a')
codex = {}

for link in links:
    href = "https:" + link['href']
    text = link.get_text(strip=True)
    if "кодекс" in text:
         codex[text] = href

In [3]:
codex

{'Гражданский кодекс (ГК РФ)': 'https://www.consultant.ru/document/cons_doc_LAW_5142/',
 'Жилищный кодекс (ЖК РФ)': 'https://www.consultant.ru/document/cons_doc_LAW_51057/',
 'Налоговый кодекс (НК РФ)': 'https://www.consultant.ru/document/cons_doc_LAW_19671/',
 'Трудовой кодекс (ТК РФ)': 'https://www.consultant.ru/document/cons_doc_LAW_34683/',
 'Уголовный кодекс (УК РФ)': 'https://www.consultant.ru/document/cons_doc_LAW_10699/',
 'Бюджетный кодекс (БК РФ)': 'https://www.consultant.ru/document/cons_doc_LAW_19702/',
 'Таможенный кодекс ЕАЭС': 'https://www.consultant.ru/document/cons_doc_LAW_215315/',
 'Арбитражный процессуальный кодекс (АПК РФ)': 'https://www.consultant.ru/document/cons_doc_LAW_37800/',
 'Земельный кодекс (ЗК РФ)': 'https://www.consultant.ru/document/cons_doc_LAW_33773/'}

In [4]:
import time
from tqdm import tqdm

codex_df = pd.DataFrame(columns=["name_codex", "url_codex", "name_article", "url_article", "content_article"])

for i, item in tqdm(enumerate(codex.items())):

    name_codex, url_codex =  item

    content_codex = requests.get(url_codex)
    soup_content_codex = BeautifulSoup(content_codex.text, 'lxml')
    articles = soup_content_codex.select("div.document-page__toc ul a")
    article_dict = {a.text.strip(): a['href'] for a in articles if "Статья" in a.text.strip()}
    
    #time.sleep(3)
    for j, x in tqdm(enumerate(article_dict.items())):
        
        name_article, url_article = x

        content_codex = requests.get(base_url+url_article)
        soup_article = BeautifulSoup(content_codex.text, 'html.parser')
        content_div = soup_article.find('div', class_='document-page__content document-page_left-padding')

        paragraphs = content_div.find_all('p')
        text_content = '\n'.join(p.get_text(strip=True) for p in paragraphs)

        new_row = {"name_codex": name_codex, "url_codex": url_codex, 
                   "name_article": name_article, "url_article": url_article,
                   "content_article": text_content}
        codex_df = pd.concat([codex_df, pd.DataFrame([new_row])], ignore_index=True)

0it [00:00, ?it/s]

592it [01:37,  6.10it/s]
243it [00:36,  6.67it/s]
268it [00:45,  5.88it/s]
530it [01:24,  6.25it/s]
528it [01:27,  6.03it/s]
406it [01:03,  6.37it/s]
465it [01:15,  6.18it/s]
418it [01:04,  6.47it/s]
178it [00:26,  6.75it/s]
9it [09:43, 64.88s/it]


In [5]:
codex_df.head()

,name_codex,url_codex,name_article,url_article,content_article
0,Гражданский кодекс (ГК РФ),https://www.consultant.ru/document/cons_doc_LA...,Статья 1. Основные начала гражданского законод...,/document/cons_doc_LAW_5142/d65fbe0a603d59546c...,ГК РФ Статья 1. Основные начала гражданского з...
1,Гражданский кодекс (ГК РФ),https://www.consultant.ru/document/cons_doc_LA...,"Статья 2. Отношения, регулируемые гражданским ...",/document/cons_doc_LAW_5142/8d8cd335130f04a703...,"ГК РФ Статья 2. Отношения, регулируемые гражда..."
2,Гражданский кодекс (ГК РФ),https://www.consultant.ru/document/cons_doc_LA...,Статья 3. Гражданское законодательство и иные ...,/document/cons_doc_LAW_5142/0c4ea7025fa5cb93e5...,ГК РФ Статья 3. Гражданское законодательство и...
3,Гражданский кодекс (ГК РФ),https://www.consultant.ru/document/cons_doc_LA...,Статья 4. Действие гражданского законодательст...,/document/cons_doc_LAW_5142/42424a8a7b50848693...,ГК РФ Статья 4. Действие гражданского законода...
4,Гражданский кодекс (ГК РФ),https://www.consultant.ru/document/cons_doc_LA...,Статья 5. Обычаи,/document/cons_doc_LAW_5142/585de80c7025af76bc...,ГК РФ Статья 5. Обычаи\n(в ред. Федеральногоза...


In [6]:
codex_df.shape

(3628, 5)

In [7]:
codex_df.to_json('./data/codex_df.json', orient='records', force_ascii=False)

In [8]:
codex_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3628 entries, 0 to 3627
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   name_codex       3628 non-null   object
 1   url_codex        3628 non-null   object
 2   name_article     3628 non-null   object
 3   url_article      3628 non-null   object
 4   content_article  3628 non-null   object
dtypes: object(5)
memory usage: 141.8+ KB
